In [59]:
%pylab inline
from scipy.optimize import bisect
import numpy as np

Populating the interactive namespace from numpy and matplotlib


In [60]:
# point mass definition
def r_t(R, M, m):
    print R, M, m
    return R*(m/(2*M))**(1/3.)

In [61]:
def mass_nfw(Mvir, R, cvir, omegam=0.3):
    
    def f(x):
        return np.log(1+x) - (x / (1+x))

    def delta_vir(omegam=0.3):
        x = omegam - 1.
        deltac = 18*np.pi**2 + 82*x -39*x**2
        return deltac/omegam

    h = 0.7
    term2 = delta_vir(omegam) * omegam / 97.2
    term3 = Mvir * h / 1e12
    rvir = (206./h) * term2**(-1./3.) * term3**(1./3.)
    rs = rvir/cvir
    x = R/rs
    return Mvir*f(x)/f(cvir)

In [62]:
mass_nfw(1e12, 250, 16.)/1e12

0.98553921515998988

In [63]:
def mass_plummer(M, R, a):
    return M*R**3/ (R**2 + a**2)**(1.5)

In [64]:
mass_plummer(2.3e11, 15., 20)/1e10

4.968

In [65]:
def mass_hernquist(M, R, a):
    return M*R**2. / (R+a)**2.

In [66]:
mass_hernquist(1e11, 200., 13.1)/1e11

0.8808320163059622

In [67]:
def r_t1(R, host, sat, Mhost, Ahost, msat, asat):
    
    def r_t1_b(r_t, R, host, sat, Mhost, Ahost, msat, asat):
        """
        R (kpc): the separation between two galaxies' COMs
        host (int): 1,2,3 which correspond to nfw, plummer, hernquist profiles
        sat (int): 1,2,3 which correspond to nfw, plummer, hernquist profiles
        Mhost (Msun): the mass of the host
        Ahost (kpc): the scale radius or concentration for the chosen host mass profile
        msat (Msun): the mass of the satellite
        asat (kpc): the scale radius or concentration for the chosen satellite mass profile

        """
        R_R = np.arange(1., R+0.1*2., 0.1)
        
        #make this part better
        if host == 1:
            MR = mass_nfw(Mhost, R_R, Ahost)
            
        if host == 2:
            MR = mass_plummer(Mhost, R_R, Ahost)
            
        if host == 3:
            MR = mass_hernquist(Mhost, R_R, Ahost)
            
        if sat == 1:
            mr = mass_nfw(msat, r_t, asat)
            
        if sat == 2:
            mr = mass_plummer(msat, r_t, asat)
            
        if sat == 3:
            mr = mass_hernquist(msat, r_t, asat)
        
        #print('host mass enclosed', MR[:-1])
        #print MR, mr
        #mr = mass_plummer(msat, r_t, asat)
        #MR = mass_nfw(Mhost, R_R, Ahost)

        dlnM_dlnR = np.gradient(list(np.log(MR)), R_R[1]-R_R[0])
        r_cut = np.argmin(np.abs(R_R-R))
        der = dlnM_dlnR[r_cut]
        f = r_t - R*(mr/MR[r_cut] * (1/(2-der)))**(1/3.0)
        return f 

    rt_find = bisect(r_t1_b, 1., R, args=(R, host, sat, Mhost, Ahost, msat, asat))
    
    return rt_find

In [72]:
M = 1.5e12
A = 9.56
a = 20.
m = 2.3e11
R = 150.

#r_t1 should be larger than r_t
r_t1(R,1, 2, M, A, m, a)#, r_t(R, M , m)

70.99232672790032

In [73]:
M = 2e12
m = 2.3e11
A = 9.36
a = 20.
R = 150.

#r_t1 should be larger than r_t
r_t1(R,1, 2, M, A, m, a)#, r_t(R, M , m)

65.75390726234136